# Documentation for the Multi-Agent Patient Diagnostics Workflow

This document outlines the five core agents in a sequential, LangGraph-orchestrated patient diagnostics system: **Audio-to-Text Agent**, **Memory Agent**, **Literature Agent**, **Decision Agent**, and **Cross-Verifying (Verification) Agent**. The Web Scraping Agent, part of the architecture, is not implemented but would handle live web data. The system leverages **Redis** for transient data, **Postgres** for persistence, **Pinecone** for vector search, **LangChain** for LLM integration, and additional tools (FastAPI, Whisper, boto3) for robustness. Below, each agent’s role, inputs, process, outputs, and technologies are summarized.

## 1. Audio-to-Text Agent

**Role**: Captures and processes doctor-patient audio conversations to extract structured medical information, acting as the workflow’s entry point.

**Inputs**: Audio file (.wav/.mp3) uploaded via FastAPI endpoint with patient ID.

**Process**:
- Stores raw audio in S3/MinIO for traceability.
- Transcribes audio using OpenAI Whisper (base model for speed; large for accuracy).
- Extracts entities (demographics, symptoms with duration/severity, context like travel history, intent) using a LangChain LLM chain with a custom prompt and few-shot example.
- Normalizes symptoms with a dictionary mapper (e.g., "high temperature" → "fever" with SNOMED-CT code; expandable to quickumls).
- Validates output and adds timestamp and S3 storage reference.

**Outputs**: JSON with:
- `patient_id`
- `demographics` (e.g., `{"age": 45, "gender": "male"}`)
- `symptoms` (e.g., `[{"name": "fever", "duration": "2 weeks", "normalized_name": "fever", "snomed_code": "386661006"}]`)
- `context` (e.g., `{"travel_history": "Recent trip to India"}`)
- `intent` (e.g., "diagnosis")
- `raw_transcript`
- `audio_storage` (S3 key)
- `timestamp`
Stored in Redis as `{patient_id}:audio_to_text`.

**Technologies**:
- **FastAPI**: API endpoint for audio uploads.
- **OpenAI Whisper**: Speech-to-text transcription.
- **LangChain**: PromptTemplate, ChatOpenAI, LLMChain for entity extraction.
- **boto3**: S3/MinIO for audio storage.
- **Redis**: Output storage.
- **Logging**: Traceability.
- **LangGraph**: Node for workflow integration.

## 2. Memory Agent

**Role**: Serves as the "clinical memory," storing new patient cases and retrieving similar past cases for diagnostic context.

**Inputs**: Structured JSON from Audio-to-Text Agent (via Redis) containing patient_id, demographics, symptoms, context, intent.

**Process**:
- Fetches input from Redis.
- Stores case in Postgres (`patient_cases` table with JSONB fields: demographics, symptoms, etc., using upsert for updates).
- Generates vector embedding of concatenated data (demographics + symptoms + context + intent) with OpenAIEmbeddings.
- Upserts embedding and metadata to Pinecone index (`patient-cases`).
- Retrieves top-5 similar cases via Pinecone cosine similarity search, excluding the current case.
- Handles errors with fallbacks and logging.

**Outputs**: JSON with:
- `patient_id`
- `similar_cases` (list of `{"similar_patient_id", "similarity_score", "metadata"}`)
- `timestamp`
Stored in Redis as `{patient_id}:memory`.

**Technologies**:
- **psycopg2**: Postgres storage.
- **Pinecone**: Vector database for case embeddings and similarity search.
- **OpenAIEmbeddings**: Embedding generation.
- **Redis**: Input/output storage.
- **Logging**: Error tracking.
- **LangGraph**: Node integration. Enhanced with self-exclusion in retrieval.

## 3. Literature Agent

**Role**: Retrieves and summarizes evidence from curated, static medical literature (e.g., guidelines, PubMed abstracts) to support diagnostics.

**Inputs**: Patient query/transcript from Audio-to-Text and similar cases from Memory (via Redis), patient_id.

**Process**:
- Fetches inputs from Redis.
- Generates query embedding (query + memory context) using OpenAIEmbeddings.
- Retrieves top-10 relevant chunks from Pinecone index (`medical-literature`).
- Optionally fetches full document for high-relevance chunks (>0.8 score) from S3.
- Summarizes chunks via LangChain LLM chain with a prompt focused on findings, guidelines, and citations.
- Validates and enhances output with relevance scores.

**Outputs**: JSON with:
- `patient_id`
- `evidence` (list of `{"snippet", "source", "relevance_score", "citation"}`)
- `summary` (concise evidence summary)
- `timestamp`
Stored in Redis as `{patient_id}:literature`.

**Technologies**:
- **Pinecone**: Vector search for literature chunks.
- **OpenAIEmbeddings**: Query/chunk embedding.
- **LangChain**: PromptTemplate, ChatOpenAI, LLMChain for summarization.
- **boto3**: S3 for full documents.
- **psycopg2**: Document metadata storage (ingestion).
- **Redis**: Input/output storage.
- **Logging**: Traceability.
- **LangGraph**: Node integration. Added ingestion function for new documents (chunking, upsert to Pinecone/Postgres/S3).

## 4. Decision Agent

**Role**: Synthesizes inputs from upstream agents to produce differential diagnoses, recommended tests, treatments, and explanations.

**Inputs**: Patient query from Audio-to-Text, similar cases from Memory, literature from Literature Agent, scraped data from Web Scraping (placeholder), patient_id (via Redis).

**Process**:
- Fetches and validates inputs from Redis (query required; fallbacks for others).
- Runs LangChain LLM chain with a prompt enforcing JSON schema, few-shot example, and missing-data handling.
- Parses and validates output, enhances with timestamp, model version, formatted explanation, and PubMed URLs.
- Stores in Postgres (`decisions` table) and Redis for Verification Agent.

**Outputs**: JSON with:
- `patient_id`
- `differential_diagnoses` (e.g., `[{"condition": "TB", "confidence": 0.82}]`)
- `recommended_tests`
- `suggested_treatment`
- `explanation`
- `evidence_sources`
- `risk_assessment` (e.g., `{"severity": "high", "urgency": "immediate"}`)
- `alternative_explanations`
- `confidence_method`
- `formatted_explanation`
- `decision_timestamp`
- `model_version`
Stored in Redis as `{patient_id}:decision`.

**Technologies**:
- **LangChain**: PromptTemplate, ChatOpenAI, LLMChain for synthesis.
- **psycopg2**: Postgres storage.
- **Redis**: Input/output storage.
- **Logging**: Error tracking.
- **LangGraph**: Node integration. Enhanced with schema validation and Markdown formatting.

## 5. Cross-Verifying (Verification) Agent

**Role**: Acts as a final quality gate, verifying the Decision Agent’s output against all upstream data for consistency, accuracy, and completeness.

**Inputs**: Query from Audio-to-Text, similar cases from Memory, literature from Literature Agent, scraped data (placeholder), Decision output (from Redis or Postgres fallback), patient_id.

**Process**:
- Fetches and validates inputs from Redis/Postgres (decision and query required).
- Runs LangChain LLM chain with a verification prompt to cross-check diagnoses, flag inconsistencies, and adjust confidences.
- Parses, validates schema, and enhances with timestamp, model version, formatted explanation.
- Stores in Postgres (`verifications` table).

**Outputs**: JSON with:
- `patient_id`
- `verified_diagnoses` (list with adjusted confidences)
- `verified_tests`
- `verified_treatment`
- `verification_explanation`
- `inconsistencies` (list of issues, e.g., "Literature contradicts TB diagnosis")
- `final_risk_assessment`
- `corrections` (suggested fixes)
- `verified` (bool)
- `confidence_method`
- `formatted_verification_explanation`
- `verification_timestamp`
- `model_version`
Stored in Postgres.

**Technologies**:
- **LangChain**: PromptTemplate, ChatOpenAI, LLMChain for verification.
- **psycopg2**: Postgres storage.
- **Redis**: Input storage.
- **Logging**: Traceability.
- **LangGraph**: Node integration. Enhanced with Postgres fallback and verified flag.

**Note**: The system assumes a Postgres schema with tables `patient_cases`, `literature_docs`, `decisions`, and `verifications`. Redis keys follow `{patient_id}:{agent}` pattern. The Web Scraping Agent, if implemented, would fetch live data and store in Redis as `{patient_id}:web_scraping` for Decision/Verification Agents.